In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
sys.path.append(module_path)

import torch
import torch as tc
from torch.func import jacfwd, vmap, grad,jacrev

#from lib.model import Model
from lib.matrixM import MatrixM


""" def f(x, y):
  return x + y ** 2
x, y = torch.randn(5), torch.randn(5)
jacobian = jacfwd(lambda x,y: x+y**2, argnums=1)(x, y)
expected = torch.diag(2 * y)
assert torch.allclose(jacobian, expected)
jacobian
 """


ModuleNotFoundError: No module named 'New_model'

In [51]:

class Model():
    
    def __init__(self, d, m, theta=None, kind='gaussian'):
        
        super().__init__()
        self.m = m       # model parameter 
        self.d = d       # dimension of b (same as x)
        
        # size of theta according to the the model
        if kind == 'gaussian':
            self.s = 2*m+d*m 
        elif kind== 'nn':
            self.L = 0.5
            self.l = 3
            self.s = m + m*d * self.l + d * self.l 
        else:
            self.s = 2*m+d*m
            warnings.warn("The type of network has not The theta parameter has not been initialized") 
        
        # Dictionary of models 
        methods = {
            'gaussian'        : self.gaussian_x,   # Eq. (16)
        }

        # Warning : une initialisation à ones fait échouer la minimisation pour theta0 !
        self.theta = tc.rand(self.s) if theta is None else theta
        #self.theta = tc.nn.Parameter(tc.rand(self.s) if theta is None else theta)
        #self.theta = tc.nn.Parameter(2.0*tc.ones(self.s) if theta is None else theta) 

        # batched derivatives , input are (d,)
        self.eval_x    = methods[kind]
        self.forward   = vmap(self.eval_x)
        self.dx        = vmap(grad(self.eval_x),             in_dims=(0,) )
        self.dxx       = vmap(grad(grad(self.eval_x)),       in_dims=(0,) )
        self.dxxx      = vmap(grad(grad(grad(self.eval_x))), in_dims=(0,) )

        # work in progress
        # self.dtheta = vmap(lambda x : autograd(self.eval_x(x), self.theta)[0])

        
    def splitter_gaussian(self):
        c = self.theta[:self.m]
        b = self.theta[self.m: self.m+self.m*self.d ].reshape(self.m,self.d)
        w = self.theta[self.m+self.m*self.d:]
        return (c,b,w)
    
    def splitter_nn(self):
        c = self.theta[:self.m]
        b=self.theta[self.m:self.m+self.d*self.l].reshape(self.l,self.d)
        w=self.theta[self.m+self.d*self.l:].reshape(self.l,self.m,self.d)
        return (c,b,w)
    
    def gaussian_x(self, x: tc.Tensor):
        """        
        Input :  a SCALAR x of shape (0,)
        Output : a SCALAR
        """
        c, b, w = self.splitter_gaussian()
        return tc.sum(c*tc.exp(-w**2 *(x-b).T**2)) 
    
    """ def forward_t(self, theta, x):
        self.theta = theta
        c,b,w = self.splitter_gaussian()
        diff = x.unsqueeze(2)-b.t().unsqueeze(0) # tensor with shape (npoints,d,m)
        distance = torch.norm(diff, dim=1)**2 # tensor of dimension (npoints, m)
        exp_factor = torch.exp(-w.unsqueeze(0)**2*distance)
        return torch.sum(c.unsqueeze(0)*exp_factor, dim=1).unsqueeze(1) """
    def forward_t(self, theta, x):
        self.theta = theta
        c,b,w = self.splitter_gaussian()
        vect=lambda x: (c*tc.exp(-w**2*(x-b)**2)).sum()
        return vect(x)
    
    def grad_theta(self, theta, samples):
        self.theta = theta
        jacobian = jacrev(lambda theta, x: self.forward_t(theta, x),  argnums=0)(theta, samples)
        return jacobian.squeeze()
    def out_prod(self,theta,x):
        out_p=lambda theta,x_: tc.outer(self.grad_theta(theta,x_),self.grad_theta(theta,x_))
        M=vmap((out_p),in_dims=(None,0))(theta,x)
        return tc.mean(M,dim=0)
        

In [62]:
import time
start = time.time()

torch.manual_seed(0)

d = 1
m = 5
NN = Model(d,m)
theta = NN.theta
samples = torch.rand(2, d)
#print(samples.T)
#print(NN.forward(samples).T)

#result_grad = NN.grad_theta(theta, samples[1])
#end = time.time()
#print(end-start)
#print(NN.forward_t(theta,samples).unsqueeze())
#print(result_grad.shape)
MatrixM=NN.out_prod(theta,samples)
print(MatrixM.shape)


torch.Size([15, 15])


In [5]:
""" tensorproduct = torch.zeros(samples.size()[0], NN.theta.size()[0], NN.theta.size()[0])
for iter in range(0, samples.size()[0]):
    tensorproduct[iter] = torch.outer(result_grad[iter], result_grad[iter]) 
    
torch.mean(tensorproduct, dim=0).shape """

torch.Size([30, 30])

In [ ]:
""" def u(theta,x):
    return x**2*theta.dot(theta)
n=10
space_x=tc.linspace(0,4,100)
theta=tc.rand(n)
def grad_u(theta,x):
    return jacrev(u,argnums=0)(theta,x)
def out_prod(theta,vect_x):
    M=lambda vect,x: tc.outer(grad_u(vect,x),grad_u(vect,x))
    return vmap((M),in_dims=(None,0))(theta,vect_x)
    
M=tc.mean(out_prod(theta,space_x),dim=0)
print(M.shape) """

In [61]:
M=MatrixM(d, m, samples)

TypeError: 'Tensor' object is not callable